<img src="images/steno3d.png" width="200"></img>
<br>
<h1 style="color:#000080;"> Volume Example: MRI Brain </h1> 

This example shows <a href="https://en.wikipedia.org/wiki/Magnetic_resonance_imaging"><b>Magnetic Resonance Image (MRI)</b></a> of head with skull partially removed. The dataset (provided by <a href=https://graphics.stanford.edu/data/voldata/><b>University of North Carolina</b></a>) consists of 109 binary files of 256 x 256 pixels. Additional description on dataset can be found <a href=https://graphics.stanford.edu/data/voldata/MRbrain.info><b>online</b></a>. Because the data are defined in 3D spatial locations on a regular grid, it is best to be visualized as volume. 

<div>
    <figure>
    <div style="height: 400px">
    <img src="images/brain1.png" style="float: left; width: 55%; margin-top: 5% ">
    <img src="images/brain2.png" style="float: right; width: 40%;margin-top: 5% ">
    </div>
    <figcaption align="middle"><div style="width: 40%; margin-left: auto; margin-right: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-top: 25%; margin-bottom: 3%"> Figure 1: Visualization of MRI Brain in Steno3D </div>
    </figcaption> 
    </figure>
</div>


<h3 style="color:#000080;"> Import Dependencies </h3> 

In [ ]:
import numpy as np
import steno3d

<h3 style="color:#000080;"> Login to Steno3D </h3> 

Before using Steno3D, you need to <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key. 

The option to `skip_credentials` prevents saving your developer key to the mybinder.org server. When working on your local computer, remove this argument and your key will be saved.

In [ ]:
steno3d.login(skip_credentials=True)

<h3 style="color:#000080;"> Steno3D Volume </h3>

Before visualizing the MRI brain data in this notebook, we need to know how we can make volumes in Steno3D. The way that Steno3D works is to first create a mesh and then bind the data to the mesh. The mesh defines the geometry of the volume. So the first step to create volume is to make what we call `Mesh3DGrid`. This mesh consists of x, y and z grid spacing with origin vector (x0) corresponding to the lower left corner of the grid which by default is equal to [0.0, 0.0, 0.0]. A simple example of `Mesh3DGrid` is shown in Figure 2.

<div>
<figure>
<img src="images/mesh3dgrid.png" alt="volume_Mesh3DGrid"  align= "middle" width="450" height="450">
  <figcaption align="middle" ><div style="width: 30%; margin-left: auto; margin-right: auto; font-weight: bold; font-style: italic; font-size: 15px; margin-top: 3%; margin-bottom: 3%">Figure 2: Volume Mesh3DGrid</div></figcaption>
</figure>
</div>

As shown in Figure1, <b>h1</b>, <b>h2</b> and <b>h3</b> are tensor arrays, flat arrays containing the width of each consecutive cell in <b>X</b>, <b>Y</b> and <b>Z</b> direction, respectively. To bind the data to volume in Steno3D we use a dictionary containg a flat array of data corresponding to cell centers ('CC'). An important thing to keep in mind when binding the data to the volume is to convert the data into a flattened array using numpy's flatten( ) function. For example, as a m x n x p array, you need to flatten it into a (mnp) x 1 array. When binding data to a volume with `Mesh3DGrid` we need to specify data order. In Steno3D, this is stored in row major (C-style) by default, but column-major ordering (Fortran-style) is also available (<a href="https://python.steno3d.com/en/latest/content/api/resources/data.html#resources-data">DataArray documentation</a>).

One significant difference between volumes and other resources (points, lines, surfaces) is that data is required. This means that for the volume to pass the validation process it is required for the data to be bounded and uploaded in Steno3D.

<h3 style="color:#000080;"> Create a Project </h3>

Before making any resources (points, lines, surfaces or volumes) in Steno3D, we need to create a `Project` to assign the resource to that project. Projects organize all your resources together and can contain a combination of resources; they can also be viewed and shared in Steno3D. 

Following are the properties of projects that you can modify if desired:

<ul>
    <li><b>title</b>: Title of the project</li>
    <li><b>description</b>: Description of the project</li>
    <li><b>public</b>: If True, the project is viewable by anyone online. If False, the project is private and only viewable by people you choose to share it with. Free and Academic accounts have limits on the number of private projects allowed; if you would like more, upgrade your account at steno3d.com.</li>
    <li><b>resources</b>: List of resources the project contains.</li>
</ul>

You can <a href="https://steno3d.com/explore">explore public projects on steno3d.com.</a> Additional <a href="https://python.steno3d.com/en/latest/content/api/projects.html">API documentation</a> is available online.

In [ ]:
MRI_brain_proj = steno3d.Project(
    title="MRI brain", 
    description="MRI of head with skull partially removed ",
    public=False
)

<h3 style="color:#000080;"> MRI Brain Data </h3>

Before making the `Volume Mesh3DGrid`, we need to read, parse and save the MRI Brain Data. As described before the data consists of 109 binary files of 256 x 256 pixels. You can download the binary files through MRI brain module from the steno3d.examples. This module downloads the data and save it to a folder in the home directory (or another location you may specify).    

In [ ]:
from steno3d.examples import Brain
Brain.fetch_data(verbose= False)  # To specify a different data directory than the
                                  # default '$HOME/.steno3d_python_assets/' use
                                  # keyword argument `directory='/path/to/your/directory/'`

In the cell below we read and down sample the MRI brain data. We down sample the data because the raw dataset is too memory intensive for Steno3D's upload limits. Currently, Steno3D has maximum capacity of 5 MB for individual resources and 25 MB for the total project. More information on maximum project size and resources is available on the <a href="https://steno3d.com/pricing">plan details page</a> . 

In [ ]:
dataall = []

for filename in Brain.filenames:            # iterate over 109 binary files 
    x = np.fromfile(filename, dtype='>i2')  # read binary data (16-bit integers)
    dataall.append(x.reshape((256, 256), order ='C'))

data = np.array(dataall).astype(float)

brain_data = data[::, -30:-210:-2, -30:-210:-2]  # down sampling data

<h3 style="color:#000080;"> Volume Mesh3DGrid </h3>

Now it is time to make a volume mesh, `Mesh3DGrid`. This mesh has the following properties:

<ul>
    <li><b>h1, h2</b> and <b>h3</b> (required)<sup><b>*</b></sup>: X, Y and Z grid spacing</li>
    <li><b>x0</b>: origin vector corresponding to the lower-left corner of the grid. The default value is [0.0, 0.0, 0.0].</li>
    <li><b>title</b></li>
    <li><b>description</b></li>
    <li><b>opts</b>: The only available option is wireframe, set to `True` or `False` with a dictionary  </li>
</ul>

(*): For the Volume resource to pass validation prior to upload, required properties must be set.

For more information on volume mesh, you can check <a href="https://python.steno3d.com/en/latest/content/api/resources/volume.html">Volume documentation</a>.

In [ ]:
# the voxel grids in MRI_Brain data are rectangular, and the 
# (X:Y:Z) aspect ratio of each grid is 1:1:2. Here due to the way we 
# parse the data, the aspect ratio of voxel grids is changed to 2:1:1.  

h1 = 2*np.ones(brain_data.shape[0])
h2 = np.ones(brain_data.shape[2])
h3 = np.ones(brain_data.shape[1]) 

MRI_brain_mesh = steno3d.Mesh3DGrid(
    h1=h1, 
    h2=h2, 
    h3=h3,
)

<h3 style="color:#000080;"> Create Volume </h3>

Now that we have created `Volume Mesh3DGrid` and defined the geometry of volume, we need to create a volume in steno3d. The properties of `Volume` are:

<ul>
    <li><b>project</b> (required): The project(s) that contain the resource </li>
    <li><b>mesh</b> (required): Volumes use `Mesh3DGrid` </li>
    <li><b>data</b> (required): A list of `DataArrays` bound to the mesh </li>
    <li><b>opts</b>: A dictionary of options</li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>

You may have noticed that unlike other resources (point, line and surface), volume requires data. This means without data, the volume cannot pass the validation process in steno3d. 

Additional <a href="https://python.steno3d.com/en/latest/content/api/resources/volume.html">API documentation</a> is available online.

In [ ]:
MRI_brain_volume = steno3d.Volume(
    project=MRI_brain_proj,
    mesh=MRI_brain_mesh, 
    title='MRI_brain Volume Resource',
)

<h3 style="color:#000080;"> Binding Volume Data</h3>

It is time to bind the data to volume resource. To do this, we need a `DataArray` which contains:

<ul>
    <li><b>array</b> (required): 1D-array of data </li>
    <li><b>order</b>: Either 'c', C-style row-major ordering (default), or 'f', Fortran-style column-major ordering </li>
    <li><b>title</b></li>
    <li><b>description</b></li>
</ul>


Then we need to bind the 'DataArray' to volume resource with a dictionary containing:

<ul>
    <li><b>location</b> (required): For volumes, this must be 'CC' (cell centers)</li>
    <li><b>data</b> (required): the DataArray </li>
</ul>

Also, more than one data set can be bound to a volume by using a list of these dictionaries.

For more information on binding the data to a volume resource, you can check <a href="https://python.steno3d.com/en/latest/content/api/resources/volume.html">API documentation</a>. 



In [ ]:
MRI_brain_data = steno3d.DataArray(
    title='MRI_brain data',
    array = brain_data.flatten(),
)


MRI_brain_volume.data = [dict(
    location='CC',
    data=MRI_brain_data,
)]

<h3 style="color:#000080;"> Upload and Plot </h3>

Calling `Upload` will check that all aspects of volume and its containing project are built correctly, then upload the project to <a href="https://steno3d.com/">steno3d.com</a>.

In [ ]:
MRI_brain_proj.upload()

In [ ]:
MRI_brain_volume.plot()